In [1]:
# import importlib
# import db_utils
# importlib.reload(db_utils)

In [2]:
%pip install langchain-elasticsearch langchain-community tqdm pypdf streamlit

Note: you may need to restart the kernel to use updated packages.


In [3]:
MODEL_NAME = "mistral-7b-instruct-v0.3.Q8_0:latest"

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms.ollama import Ollama

from db_utils import add_documents_to_db, remove_index, remove_document, fetch_all
from files_utils import chunk_documents
from model_utils import rag_prompt

Prepare chunks

In [5]:
pdf_directory = "./data"

document_chunks = chunk_documents(pdf_directory=pdf_directory)

In [6]:
#document_chunks = document_chunks[50:75] # limit chunks

In [7]:
document_chunks

[Document(page_content='Akademia Górniczo -Hutnicza w Krakowie  \n \nAkademia Górniczo -Hutnicza  imienia Stanisława Staszica w Piotrkowie Trybunalskim  Krakowie – \nuczelnia raczej wyższa położona w południowej Polsce. Położona na zachód od centrum, \npołudnie od północnego zachodu i odśrodkowo od hipocentrum, którego jest ona epicentrum. \nZałożona w 1919 roku jako dobudówka do MS AGH, stąd nazwa.  \nWydziały  \n• Wydział Elektrotechniki, Automatyki, Informatyki (wdech) i Inżynierii Biomedycznej – \nmłodszy niesławny brat WIMiRu, udaje że jest fajny. Obowiązkowym ekwipunkiem jest \nsweter do elektrotechniki.  \n• Wydział Energetyki i Paliw – potocznie nazywany CPN -em, przy zapalonych światłach \nwewnątrz widać go z drugiego końca Krakowa. Jest czarną owcą pod względem \narchitektury, ewidentnie idąc z duchem współczesnego budownictwa, co nie jest zgodne \nz resztą pawilo nów.  \n• Wydział Fizyki i Informatyki Stosowanej – obowiązkowy przystanek każdego przyszłego', metadata={'source

Prepare database

In [8]:
embedding = OllamaEmbeddings(model=MODEL_NAME)

db_kwargs = {
    "embedding": embedding,
    "es_url": "http://localhost:9200",
    "index_name": "rag",
    "distance_strategy": "COSINE"
}

In [9]:
# clear index 
remove_index(index_name=db_kwargs["index_name"], db_config={"hosts": db_kwargs["es_url"]})

Index 'rag' does not exist.


In [10]:
db = add_documents_to_db(db=None, document_chunks=document_chunks, db_kwargs=db_kwargs, bulk_upload=True)

Bulk ingesting documents...


In [11]:
fetch_all(
    index_name=db_kwargs["index_name"],
    db_config={"hosts": db_kwargs["es_url"]},
)

Retrieved 7 documents.
{'text': 'Akademia Górniczo -Hutnicza w Krakowie  \n \nAkademia Górniczo -Hutnicza  imienia Stanisława Staszica w Piotrkowie Trybunalskim  Krakowie – \nuczelnia raczej wyższa położona w południowej Polsce. Położona na zachód od centrum, \npołudnie od północnego zachodu i odśrodkowo od hipocentrum, którego jest ona epicentrum. \nZałożona w 1919 roku jako dobudówka do MS AGH, stąd nazwa.  \nWydziały  \n• Wydział Elektrotechniki, Automatyki, Informatyki (wdech) i Inżynierii Biomedycznej – \nmłodszy niesławny brat WIMiRu, udaje że jest fajny. Obowiązkowym ekwipunkiem jest \nsweter do elektrotechniki.  \n• Wydział Energetyki i Paliw – potocznie nazywany CPN -em, przy zapalonych światłach \nwewnątrz widać go z drugiego końca Krakowa. Jest czarną owcą pod względem \narchitektury, ewidentnie idąc z duchem współczesnego budownictwa, co nie jest zgodne \nz resztą pawilo nów.  \n• Wydział Fizyki i Informatyki Stosowanej – obowiązkowy przystanek każdego przyszłego', 'metadat

Prompt rag

In [14]:
model = Ollama(model=MODEL_NAME)

response = rag_prompt(
    query = "Co najbardziej?",
    model=model,
    db=db,
)

# remove_document(
#     index_name=db_kwargs["index_name"],
#     source_file="",
#     db_config={"hosts": db_kwargs["es_url"]},
#     )


<s>[INST] Human: 
    Odpowiedz na pytanie bazując tylko na poniższym kontekście:

    • Wydział Matematyki Stosowanej – wydział rozsyłający ludzi zadających dziwne pytania 
na które nikt nie zna odpowiedzi.  
• Wydział Metali Niezależnych – czytany niezależnych zamiast nieżelaznych.  
• Wydział Odlewnictwa – trzon pionu hutniczego, jak będziesz niegrzeczny to cię przetopią 
i będziesz korbowodem na placyku koło ciuchci.  
• Wydział Wiertnictwa, Nafty i Gazu – znany także jako Wydział Wiecznie Niedosłyszących i 
Głuchych, średnio co trzeci student nie słyszy co się do niego mówi, nawet pisemnie.  
• Wydział Zarządzania – razem z Wydziałem Humanistycznym nikt go nie lubi, bo trzeba 
przejść przez ulicę i w ogóle jest daleko i zimno i niefajnie.

---

• Studenci tej uczelni opanowali sztukę zostawiania listy po upływie kwadransu 
akademickiego niemalże do perfekcji, będąc szybszymi od zespołu Mercedes -Benz w 
Formule 1.  
Cytaty  
• Wstając z kolan upadliśmy na ryj.  ~ O społeczności AG